In [2]:
import islpy as isl
from src.io_tools import *
from src.graph_tools import *
from src.swap_tools import *
from prettytable import PrettyTable
import time

In [3]:

disconnection_map = isl.Map("{ [1]->[2] ; [1]->[3] ; [3]->[4]}")
access1 = isl.Map("{ [i0] -> [i0] : 0 <= i0 <= 2; [i0] -> [-3 + i0] : 3 <= i0 <= 4; [5] -> [3] }")
access2 = isl.Map("{  [i0] -> [4] : 3 <= i0 <= 5; [i0] -> [2] : 0 <= i0 <= 1; [2] -> [3] }")

transform_map = isl.Map(" { [[a,b] -> [c]] -> [[a] -> [b,c]]}").coalesce()


In [6]:
access1.min_val()

AttributeError: 'islpy._isl.Map' object has no attribute 'min_val'

Set("{ [i0 = 1] }")

In [29]:
access1.flatten()

Map("{ [i0] -> [i0] : 0 <= i0 <= 2; [i0] -> [-3 + i0] : 3 <= i0 <= 4; [i0 = 5] -> [3] }")

In [30]:
access1.domain_map().apply_range(access2).flatten().wrap().apply(transform_map).unwrap()


Map("{ [i0] -> [-3 + i0, 4] : 3 <= i0 <= 4; [i0] -> [i0, 2] : 0 <= i0 <= 1; [5] -> [3, 4]; [2] -> [2, 3] }")

In [58]:


# [10 -> 2] -> 3

# 10 -> [2->3]

Map("{ [i0] -> [[-3 + i0] -> [4]] : 3 <= i0 <= 4; [i0] -> [[i0] -> [2]] : 0 <= i0 <= 1; [5] -> [[3] -> [4]]; [2] -> [[2] -> [3]] }")

Map("{ [[[a] -> [b]] -> [c]] -> [[a] -> [[b] -> [c]]] }")

In [18]:
project_onto_map()

project_out()

project_out_all_params()

project_out_except()

project_out_param_id()

project_out_param_id_list()



TypeError: project_out(): incompatible function arguments. The following argument types are supported:
    1. project_out(self, type: islpy._isl.dim_type, first: int, n: int) -> object
    2. project_out(self, arg0: islpy._isl.dim_type, arg1: int, arg2: int, /) -> object

Invoked with types: islpy._isl.Map

In [ ]:
disconnected_edges = " [1, 2] ; [3 , 4]"


In [2]:
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger('stevedore.extension').setLevel(logging.CRITICAL)


In [3]:
json_file_path = 'benchmarks/polyhedral/queko-bss-16qbt/16QBT_100CYC_QSE_0.json'

data = json_file_to_isl(json_file_path)

In [4]:

domain, access, schedule = filter_multi_qubit_gates(data["domain"], data["read_dependencies"], data["schedule"])


In [5]:
regex = re.compile(r'(?<=\[)(.*?)(?=\])')
num_qubits = int(regex.findall(data['read_dependencies'].range().as_set().lexmax().to_str())[0]) + 1

In [6]:
physical_qubits_domain = isl.Set(f"{{ [i] : 1 <= i <= {num_qubits} }}")

In [7]:
# Generate a 2D grid for 200 qubits (10 rows and 20 columns)
def generate_2d_grid(num_rows, num_cols):
    num_qubits = num_rows * num_cols
    graph = [[] for _ in range(num_qubits)]
    
    for i in range(num_rows):
        for j in range(num_cols):
            index = i * num_cols + j
            # Connect to the left neighbor
            if j > 0:
                graph[index].append(index - 1)
            # Connect to the right neighbor
            if j + 1 < num_cols:
                graph[index].append(index + 1)
            # Connect to the top neighbor
            if i > 0:
                graph[index].append(index - num_cols)
            # Connect to the bottom neighbor
            if i + 1 < num_rows:
                graph[index].append(index + num_cols)
    
    return graph

num_rows = 4
num_cols = 4
graph = generate_2d_grid(num_rows, num_cols)



In [8]:
backend_disconnected_edges =  extract_edges_map(graph)
shortest_paths = extract_shortest_paths(graph)

In [9]:
initial_mapping = isl.Map(f"{{ q[i] -> [i] : 1<=i<={num_qubits} }}")

In [10]:
mapping = initial_mapping
swap_count = 0
programme = schedule


In [13]:
map1 = isl.Map("{[1] -> [2]; [2] -> [5]}")
map2 = isl.Map("{[1] -> [3]; [2] -> [6]}")

map1.range_product(map2)


Map("{ [2] -> [[5] -> [6]]; [1] -> [[2] -> [3]] }")

In [11]:


iteration = 0
start = time.time()
while not programme.is_empty():
    #print(f"Iteration {iteration}")
    #print(programme.domain())
    iteration += 1
    programme_mapped = programme.apply_range(mapping)
    first_programme_mapped = programme_mapped.lexmin()
    second_programme_mapped = programme_mapped.subtract(first_programme_mapped)
    programme_access = first_programme_mapped.flat_range_product(second_programme_mapped)
    first_disconnected_edge = programme_access.intersect_range(backend_disconnected_edges).domain().lexmin()
    if first_disconnected_edge.is_empty():
        break

    disconected_qubits = programme_mapped.intersect_domain(first_disconnected_edge).range().as_set()
    q1 , q2 = disconected_qubits.dim_min_val(0).to_python(),disconected_qubits.dim_max_val(0).to_python()
    swap_count += shortest_paths[q1]['costs'][q2] -1
    new_domain = programme.domain().as_set().lex_gt_set(first_disconnected_edge.as_set()).domain()
    programme = programme.intersect_domain(new_domain).coalesce()
    swap_map = swaps_to_isl_map(shortest_paths[q1]['paths'][q2])
    mapping = apply_swaps_to_logical_qubits_map(swap_map,mapping,physical_qubits_domain)
    

end = time.time()



KeyboardInterrupt: 

In [277]:

start = time.time()
iteration = 0
while not programme.is_empty():
    iteration += 1
    programme_mapped = programme.apply_range(mapping)
    first_programme_mapped = programme_mapped.lexmin()
    second_programme_mapped = programme_mapped.subtract(first_programme_mapped)
    programme_access = first_programme_mapped.flat_range_product(second_programme_mapped)

    first_disconnected_edge = programme_access.intersect_range(backend_disconnected_edges).domain().lexmin()

    if first_disconnected_edge.is_empty():
        break

    disconected_qubits = programme_mapped.intersect_domain(first_disconnected_edge).range().as_set()
    q1 , q2 = disconected_qubits.dim_min_val(0).to_python(),disconected_qubits.dim_max_val(0).to_python()
    swap_count += shortest_paths[q1]['costs'][q2] - 1
    print(first_disconnected_edge)
    new_domain = programme.domain().as_set().lex_gt_set(first_disconnected_edge.as_set()).domain().coalesce()
    programme = programme.intersect_domain(new_domain)
    mapping = apply_swaps_to_logical_qubits_map(shortest_paths[q1]['isl_maps'][q2],mapping,physical_qubits_domain)

print(f"Total number of swaps: {swap_count}")
print(f"Total number of iterations: {iteration}")
print(f"Execution time: {time.time() - start}")


{ [10] }
{ [21] }
{ [23] }
{ [35] }
{ [36] }
{ [57] }
{ [58] }
{ [59] }
{ [66] }
{ [67] }
{ [68] }
{ [87] }
{ [88] }
{ [90] }
{ [99] }
{ [102] }
{ [112] }
{ [119] }
{ [121] }
{ [133] }
{ [134] }
{ [143] }
{ [145] }
{ [146] }
{ [152] }
{ [153] }
{ [154] }
{ [167] }
{ [168] }
{ [189] }
{ [191] }
{ [212] }
{ [213] }
{ [215] }
{ [236] }
{ [237] }
{ [252] }
{ [263] }
{ [272] }
{ [273] }
{ [274] }
{ [275] }
{ [284] }
{ [285] }
{ [286] }
{ [287] }
{ [297] }
{ [298] }
{ [306] }
{ [307] }
{ [308] }
{ [320] }
{ [321] }
{ [322] }
{ [332] }
{ [356] }
{ [363] }
{ [365] }
{ [366] }
{ [376] }
{ [377] }
{ [385] }
{ [386] }
{ [387] }
{ [396] }
{ [397] }
{ [398] }
{ [422] }
{ [432] }
{ [433] }
{ [442] }
{ [443] }
{ [444] }
{ [451] }
{ [452] }
{ [453] }
{ [454] }
{ [461] }
{ [463] }
{ [464] }
{ [472] }
{ [474] }
{ [485] }
{ [487] }
{ [498] }
{ [499] }
{ [508] }
{ [509] }
{ [510] }
{ [515] }
{ [516] }
{ [519] }
{ [530] }
{ [531] }
{ [537] }
{ [540] }
{ [551] }
{ [553] }
{ [564] }
{ [565] }
{ [576] }
{ [57

In [289]:
swap_count

151

In [290]:
iteration

110

In [16]:
results_table = PrettyTable()

results_table.field_names = ["Method","Swap Count", "Execution Time (s)"]
results_table.add_row(["Our solution", swap_count, end-start])

----------------------

In [17]:
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import CouplingMap, PassManager ,Layout
from qiskit.transpiler.passes import LookaheadSwap, SabreSwap
from qiskit_aer import Aer

In [18]:
qasm_code = json_file_qasm(json_file_path)
qc = QuantumCircuit.from_qasm_str(qasm_code)

num_qubits = qc.num_qubits

In [20]:
isl.Set('{ [[1]->[2]] ; [[2]->[5]] }')

Set("{ [[2] -> [5]]; [[1] -> [2]] }")

In [21]:
isl.Set(f"{{  [[i] -> [j]] : 1 <= i<j <= {len(graph)} }}").subtract(isl.Set('{ [[1]->[2]] ; [[2]->[5]] }'))

Set("{ [[i] -> [j]] : i > 0 and j > i and 3 <= j <= 16 and (j <= 4 or j >= 6); [[i = 1] -> [j = 5]]; [[i] -> [j = 5]] : 3 <= i <= 4 }")

In [20]:
edges = [(i, j) for i, neighbors in enumerate(graph) for j in neighbors if i < j]

coupling_map = CouplingMap(edges)

In [21]:
layout_dict = {qc.qubits[i]: i for i in range(num_qubits)}  
layout = Layout(layout_dict)

In [22]:
pass_manager = PassManager()
pass_manager.append(LookaheadSwap(coupling_map))

In [23]:
qc.count_ops()

OrderedDict([('x', 816), ('cx', 320)])

In [24]:
start_time = time.time()
transpiled_circuit = transpile(qc, coupling_map=coupling_map, initial_layout=layout)
optimized_circuit = pass_manager.run(transpiled_circuit)
end_time = time.time()

In [25]:
swap_count = sum(1 for op in transpiled_circuit.data if op[0].name == 'swap')

In [26]:
results_table.add_row(["LookaheadSwap", swap_count, end_time-start_time])

In [27]:
pass_manager = PassManager()
pass_manager.append(SabreSwap(coupling_map))

start_time = time.time()
transpiled_circuit = transpile(qc, coupling_map=coupling_map, initial_layout=layout)
optimized_circuit = pass_manager.run(transpiled_circuit)
end_time = time.time()

swap_count = sum(1 for op in transpiled_circuit.data if op[0].name == 'swap')

results_table.add_row(["SabreSwap", swap_count, end_time-start_time])

In [28]:
start_time = time.time()
transpiled_circuit = transpile(qc, coupling_map=coupling_map, initial_layout=layout,optimization_level=0)
end_time = time.time()
swap_count = sum(1 for op in transpiled_circuit.data if op[0].name == 'swap')

results_table.add_row(["without optimizer", swap_count, end_time-start_time])

In [29]:
print(results_table)

+-------------------+------------+---------------------+
|       Method      | Swap Count |  Execution Time (s) |
+-------------------+------------+---------------------+
|    Our solution   |    151     |   38.8696825504303  |
|   LookaheadSwap   |     30     |  0.5446712970733643 |
|     SabreSwap     |     22     |  0.1252748966217041 |
| without optimizer |    128     | 0.07019400596618652 |
+-------------------+------------+---------------------+
